In [25]:
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import VisualFeatureTypes
from msrest.authentication import CognitiveServicesCredentials
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import requests
from PIL import Image
from io import BytesIO
from IPython.display import display
import pandas as pd
import numpy as np

## Scraping Code

In [ ]:
from argparse import ArgumentParser
from glob import glob
from os.path import expanduser
from platform import system
from sqlite3 import OperationalError, connect

try:
    from instaloader import ConnectionException, Instaloader
except ModuleNotFoundError:
    raise SystemExit("Instaloader not found.\n  pip install [--user] instaloader")


def get_cookiefile():
    default_cookiefile = {
        "Windows": "~/AppData/Roaming/Mozilla/Firefox/Profiles/*/cookies.sqlite",
        "Darwin": "~/Library/Application Support/Firefox/Profiles/*/cookies.sqlite",
    }.get(system(), "~/.mozilla/firefox/*/cookies.sqlite")
    cookiefiles = glob(expanduser(default_cookiefile))
    if not cookiefiles:
        raise SystemExit("No Firefox cookies.sqlite file found. Use -c COOKIEFILE.")
    return cookiefiles[0]


def import_session(cookiefile, sessionfile):
    print("Using cookies from {}.".format(cookiefile))
    conn = connect(f"file:{cookiefile}?immutable=1", uri=True)
    try:
        cookie_data = conn.execute(
            "SELECT name, value FROM moz_cookies WHERE baseDomain='instagram.com'"
        )
    except OperationalError:
        cookie_data = conn.execute(
            "SELECT name, value FROM moz_cookies WHERE host LIKE '%instagram.com'"
        )
    instaloader = Instaloader(max_connection_attempts=1)
    instaloader.context._session.cookies.update(cookie_data)
    username = instaloader.test_login()
    if not username:
        raise SystemExit("Not logged in. Are you logged in successfully in Firefox?")
    print("Imported session cookie for {}.".format(username))
    instaloader.context.username = username
    instaloader.save_session_to_file(sessionfile)


if __name__ == "__main__":
    p = ArgumentParser()
    p.add_argument("-c", "--cookiefile")
    p.add_argument("-f", "--sessionfile")
    args = p.parse_args()
    try:
        import_session(args.cookiefile or get_cookiefile(), args.sessionfile)
    except (ConnectionException, OperationalError) as e:
        raise SystemExit("Cookie import failed: {}".format(e))

In [ ]:
import instaloader
from instaloader import Profile
L = instaloader.Instaloader()

try:
    L.load_session_from_file("santhosh_ramkumar", r"C:\Users\rsk29\AppData\Roaming\jupyter\runtime\kernel-afc789f5-06ee-4008-b1a2-33e81d789c97.json") #1st argument should be username and 2nd argument should be the saved file location
    print("Logged In Successfully")
except Exception as e:
    print("Error", e)

In [ ]:
import csv
# Get the Instagram profile
posts = instaloader.Profile.from_username(L.context, 'netflix').get_posts()
counter = 0
data = []

# Open the CSV file for writing
with open('comments.csv', 'w', newline='') as f:
    writer = csv.writer(f)

    # Write the header row
    writer.writerow(['post_id', 'comments'])

    # Iterate over the posts and extract the comments
    for post in posts:
        counter+=1
        comments = post.get_comments()

        # Write the comments to the CSV file
        for comment in comments:
            
        
        if counter>=2:
            break


In [ ]:
import csv
import pandas as pd

# Get the Instagram profile
posts = instaloader.Profile.from_username(L.context, 'netflix').get_posts()
counter = 0
data = []

post_lvl_df = pd.DataFrame(columns=['post_link', 'comment_count','likes_count','caption','is_video','image_link'])

# Iterate over the posts and extract the comments
for post in posts:
    post_id = post.shortcode
    counter+=1
    shortcode = post.shortcode
    post_link = 'https://instagram.com/p/'+shortcode+'/'
    likes = post.likes
    comments = post.comments
    caption = post.caption
    video = post.is_video
    image_link = post.url
    
    post_lvl_df = pd.concat([post_lvl_df, pd.DataFrame({'post_link':post_link, 'comment_count':comments, 'likes_count':likes, 'caption':caption,'image_link':image_link,'is_video':video}, index=[counter])], ignore_index=True)

    comments = post.get_comments()
    
    if(video == False):
        cmt_ctr = 0
        for comment in comments:
            data.append([post_id, comment.text])
            cmt_ctr+=1
            print('Post Number: ', counter, ' | Comment Number: ',cmt_ctr)
            if cmt_ctr>=50:
                break
            
    if counter>=1000:
        break



In [ ]:
df = pd.DataFrame(data, columns=['Post ID', 'Comments'])
df.to_csv(r'C:\Users\rsk29\Documents\UT\Fall\AUD\Project\Netflix_posts_comments.csv')
post_lvl_df.to_csv(r'C:\Users\rsk29\Documents\UT\Fall\AUD\Project\Netflix_posts_info.csv')

In [3]:
key = 'c487f19658044ff282bc8cfe798f7fee'
endpoint = 'https://image-analysis-longhorns.cognitiveservices.azure.com/'

In [4]:
computervision_client = ComputerVisionClient(endpoint, CognitiveServicesCredentials(key))

## Sentiment Analysis

In [ ]:
import pandas as pd
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords

In [ ]:
nltk.download('vader_lexicon')

### Sentiment Analysis for Netflix

In [ ]:
# Initialize NLTK's stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

# Initialize the VADER sentiment analyzer
sia = SentimentIntensityAnalyzer()

# Read the Netflix dataset
netflix_df = pd.read_csv('Netflix_Comment_Column_vF.csv')

# Perform sentiment analysis on each comment, remove stopwords, and allocate average scores based on post_id
sentiment_scores = []
average_sentiment_scores = []

for post_id, comment in zip(netflix_df['post_id'], netflix_df['Comments']):
    if isinstance(comment, float):
        # Handle float values (e.g., NaN) by converting them to an empty string
        comment = str(comment)
    
    # Tokenize the comment and remove stopwords
    words = comment.split()
    filtered_words = [word for word in words if word.lower() not in stop_words]
    filtered_comment = ' '.join(filtered_words)
    
    sentiment = sia.polarity_scores(filtered_comment)
    sentiment_scores.append(sentiment)

# Create a DataFrame for average sentiment scores
sentiment_df = pd.DataFrame(sentiment_scores)

merged_df = pd.concat([netflix_df, sentiment_df], axis=1)

def transform_to_0_1(score):
    return (score + 1) / 2

# Apply the transformation to the sentiment scores in the DataFrame
merged_df['sentiment_score'] = merged_df['compound'].apply(transform_to_0_1)

# Display the DataFrame with sentiment scores
#display(merged_df.head(5))
#print(len(merged_df))

In [ ]:
merged_df.to_csv('comments_sentiment.csv', index=False)

In [ ]:
# Group by 'Post ID' and calculate the average 'compound' score for each post
netflix_sentiment = merged_df.groupby('post_id')['sentiment_score'].mean().reset_index()

# Display the DataFrame with average compound scores
#display(netflix_sentiment .head(5))
#print(len(netflix_sentiment))

### Normalizing Netflix scores

In [ ]:
# Read the Netflix dataset
netflix_full_df = pd.read_csv('Netflix_All_Input_vF.csv')
netflix_full_df = netflix_full_df.sort_values(by='likes_count', ascending=False)
netflix_full_df = netflix_full_df.iloc[4:]
#print(len(netflix_full_df))

In [ ]:
# Calculate the minimum and maximum likes
min_likes = netflix_full_df['likes_count'].min()
max_likes = netflix_full_df['likes_count'].max()

#print(min_likes)
#print(max_likes)

# Apply Min-Max scaling to create the normalized_likes column
netflix_full_df['normalized_likes'] = ((netflix_full_df['likes_count'] - min_likes) / (max_likes - min_likes))

#display(netflix_full_df.head(5))

In [ ]:
netflix_merged_df = netflix_full_df.merge(netflix_sentiment, on="post_id", how="inner")
# Round the "normalized_likes" and "sentiment_score" columns to 4 decimal places
netflix_merged_df['normalized_likes'] = netflix_merged_df['normalized_likes'].round(4)
netflix_merged_df['sentiment_score'] = netflix_merged_df['sentiment_score'].round(4)
netflix_merged_df['compound_score'] = netflix_merged_df['sentiment_score']*netflix_merged_df['normalized_likes']
netflix_merged_df['compound_score'] = netflix_merged_df['compound_score'].round(4)

#display(netflix_merged_df.head(5))
#print(len(netflix_merged_df))

In [ ]:
# Sort the DataFrame based on the "normalized_likes" column in descending order
netflix_merged_df = netflix_merged_df.sort_values(by='normalized_likes', ascending=False)

In [ ]:
netflix_merged_df.to_csv('netflix_merged_df.csv', index=False)

### Sentiment Analysis for Disney+

In [ ]:
# Initialize the VADER sentiment analyzer
sia_disney = SentimentIntensityAnalyzer()

# Read the Netflix dataset
disney_df = pd.read_csv('DisneyPlus_Comment_Column_vF.csv')

# Perform sentiment analysis on each comment, remove stopwords, and allocate average scores based on post_id
sentiment_scores_disney = []
average_sentiment_scores_disney = []

for post_id, comment in zip(disney_df['post_id'], disney_df['Comments']):
    if isinstance(comment, float):
        # Handle float values (e.g., NaN) by converting them to an empty string
        comment = str(comment)
    
    # Tokenize the comment and remove stopwords
    words = comment.split()
    filtered_words = [word for word in words if word.lower() not in stop_words]
    filtered_comment = ' '.join(filtered_words)
    
    sentiment_disney = sia_disney.polarity_scores(filtered_comment)
    sentiment_scores_disney.append(sentiment_scores)

# Create a DataFrame for average sentiment scores
sentiment_df_disney = pd.DataFrame(sentiment_scores)

merged_df_disney = pd.concat([disney_df, sentiment_df_disney], axis=1)

# Display the DataFrame with sentiment scores
#display(merged_df_disney.head(5))

def transform_to_0_1(score):
    return (score + 1) / 2

# Apply the transformation to the sentiment scores in the DataFrame
merged_df_disney['sentiment_score'] = merged_df_disney['compound'].apply(transform_to_0_1)

# Display the DataFrame with sentiment scores
#display(merged_df_disney.head(5))
#print(len(merged_df_disney))

In [ ]:
# Group by 'Post ID' and calculate the average 'compound' score for each post
disney_sentiment = merged_df_disney.groupby('post_id')['sentiment_score'].mean().reset_index()

# Rename the 'compound' column to 'average_compound'
disney_sentiment.rename(columns={'compound': 'sentiment_score'}, inplace=True)

# Display the DataFrame with average compound scores
#display(disney_sentiment.head(5))
#print(len(disney_sentiment))

### Normalizing Disney+ scores

In [ ]:
# Read the Netflix dataset
disney_full_df = pd.read_csv('DisneyPlus_All_Input_vF.csv')
disney_full_df = disney_full_df.sort_values(by='likes_count', ascending=False)
disney_full_df = disney_full_df.iloc[6:]
print(len(disney_full_df))

In [ ]:
# Calculate the minimum and maximum likes
min_likes_disney = disney_full_df['likes_count'].min()
max_likes_disney = disney_full_df['likes_count'].max()

print(min_likes_disney)
print(max_likes_disney)

# Apply Min-Max scaling to create the normalized_likes column
disney_full_df['normalized_likes'] = ((disney_full_df['likes_count'] - min_likes_disney) / (max_likes_disney - min_likes_disney))

display(disney_full_df.head(5))

In [ ]:
disney_merged_df = disney_full_df.merge(disney_sentiment, on="post_id", how="inner")
# Round the "normalized_likes" and "sentiment_score" columns to 4 decimal places
disney_merged_df['normalized_likes'] = disney_merged_df['normalized_likes'].round(4)
disney_merged_df['sentiment_score'] = disney_merged_df['sentiment_score'].round(4)
disney_merged_df['compound_score'] = disney_merged_df['sentiment_score']*disney_merged_df['normalized_likes']
disney_merged_df['compound_score'] =disney_merged_df['compound_score'].round(4)

display(disney_merged_df.head(5))
print(len(disney_merged_df))

In [ ]:
# Sort the DataFrame based on the "normalized_likes" column in descending order
disney_merged_df = disney_merged_df.sort_values(by='normalized_likes', ascending=False)

In [ ]:
disney_merged_df.to_csv('disney_merged_df.csv', index=False)

## Cloud Vision API code to extract labels and safe-search scores

In [ ]:
!pip install google-cloud-vision
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "C:/Users/rajku/Downloads/aud-project-401401-d2fa43dc76d7.json"
import csv
from google.cloud import vision_v1
# Initialize the Vision API client
client = vision_v1.ImageAnnotatorClient()
# Function to analyze an image and return labels/topics and safe search results
def analyze_image(image_url):
    # Create an image object
    image = vision_v1.Image()
    image.source.image_uri = image_url
    # Define features for label detection and safe search analysis
    label_feature = vision_v1.Feature(type_=vision_v1.Feature.Type.LABEL_DETECTION)
    safe_search_feature = vision_v1.Feature(type_=vision_v1.Feature.Type.SAFE_SEARCH_DETECTION)
    # Create a request object
    request = vision_v1.AnnotateImageRequest(
        image=image,
        features=[label_feature, safe_search_feature]
    )
    # Perform the annotation request
    response = client.annotate_image(request)
    # Extract labels/topics
    labels = [label.description for label in response.label_annotations]
    # Extract safe search results
    safe_search = response.safe_search_annotation
    return labels, safe_search
# Function to process a CSV file containing image URLs and store results in a new CSV file
def process_csv(csv_file, output_csv_file):
    results = []
    a = 1;
    with open(csv_file, newline="", encoding="utf-8") as csvfile:
        reader = csv.reader(csvfile)
        next(reader)  # Skip the header row if present
        for row in reader:
            post_link = row[0]
            image_url = row[1]  # Assuming the image URLs are in the second column
            labels, safe_search = analyze_image(image_url)
            a = a + 1
            print(a)
            result = {
                "Post URL": post_link,
                "Image URL": image_url,
                "Labels/Topics": ", ".join(labels),
                "Safe Search Adult": safe_search.adult.name,
                "Safe Search Medical": safe_search.medical.name,
                "Safe Search Spoof": safe_search.spoof.name,
                "Safe Search Violence": safe_search.violence.name,
                "Safe Search Racy": safe_search.racy.name,
            }
            results.append(result)
            print(a)
    # Save results to a new CSV file
    with open(output_csv_file, mode='w', newline='', encoding="utf-8") as output_file:
        fieldnames = ["Post URL", "Image URL", "Labels/Topics", "Safe Search Adult", "Safe Search Medical", "Safe Search Spoof", "Safe Search Violence", "Safe Search Racy"]
        writer = csv.DictWriter(output_file, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(results)
# Specify the path to your CSV file containing image URLs
csv_file_path = "DisneyPlus_Image_URLs_vF.csv"
# Specify the path to the output CSV file
output_csv_file_path = "DisneyPlus_Image_URLs_Output_vF.csv"
# Call the function to process the CSV file and store results in a new CSV file
process_csv(csv_file_path, output_csv_file_path)
print(f"Results have been saved to {output_csv_file_path}")

## Netflix and Disney+ Analysis

In [5]:
df = pd.read_csv('Netflix_Image_URLs_Output_vF.csv')
df.head()

,Post URL,Image URL,Labels/Topics,Safe Search Adult,Safe Search Medical,Safe Search Spoof,Safe Search Violence,Safe Search Racy
0,https://instagram.com/p/CyOgBu4u6ek/,https://scontent-dfw5-1.cdninstagram.com/v/t51...,"Forehead, Nose, Face, Cheek, Skin, Lip, Chin, ...",UNLIKELY,UNLIKELY,UNLIKELY,UNLIKELY,POSSIBLE
1,https://instagram.com/p/CyLr35yJbc3/,https://scontent-dfw5-2.cdninstagram.com/v/t51...,"Water, Cloud, Sky, People in nature, Blue, Fla...",VERY_UNLIKELY,VERY_UNLIKELY,VERY_UNLIKELY,VERY_UNLIKELY,UNLIKELY
2,https://instagram.com/p/CyLj5QBucUt/,https://scontent-dfw5-2.cdninstagram.com/v/t51...,"Eyelash, Flash photography, Poster, Art, Font,...",UNLIKELY,VERY_UNLIKELY,VERY_UNLIKELY,UNLIKELY,POSSIBLE
3,https://instagram.com/p/CyD4BLvRppW/,https://scontent-dfw5-1.cdninstagram.com/v/t51...,"Forehead, Nose, Chin, Eyebrow, Jaw, Beard, Eye...",VERY_UNLIKELY,UNLIKELY,VERY_LIKELY,UNLIKELY,VERY_UNLIKELY
4,https://instagram.com/p/CyCN8YSSBIj/,https://scontent-dfw5-2.cdninstagram.com/v/t51...,"Water, Sky, Cloud, Shorts, Body of water, Fun,...",UNLIKELY,VERY_UNLIKELY,UNLIKELY,UNLIKELY,POSSIBLE


In [6]:
df_all_data = pd.read_csv('Netflix_All_Input_vF.csv')
df_all_data['post_id'] = df_all_data['post_link'].str.rsplit('/', 2).str[-2]
df_all_data.head()

C:\Users\vaish\AppData\Local\Temp\ipykernel_61528\1255580430.py:2: FutureWarning: In a future version of pandas all arguments of StringMethods.rsplit except for the argument 'pat' will be keyword-only.
  df_all_data['post_id'] = df_all_data['post_link'].str.rsplit('/', 2).str[-2]


,Unnamed: 0,post_link,comment_count,likes_count,caption,is_video,image_link,post_id
0,5,https://instagram.com/p/CyOgBu4u6ek/,1618,279051,"ah, yes, romance\n\n📺: fair play",False,https://scontent-dfw5-1.cdninstagram.com/v/t51...,CyOgBu4u6ek
1,10,https://instagram.com/p/CyLr35yJbc3/,2221,508557,The sixth and final season of The Crown is com...,False,https://scontent-dfw5-2.cdninstagram.com/v/t51...,CyLr35yJbc3
2,11,https://instagram.com/p/CyLj5QBucUt/,463,126168,"To the house of Usher, whose time has come. 4 ...",False,https://scontent-dfw5-2.cdninstagram.com/v/t51...,CyLj5QBucUt
3,20,https://instagram.com/p/CyD4BLvRppW/,157,22146,daily affirmation courtesy of milton\n\nNEW EP...,False,https://scontent-dfw5-1.cdninstagram.com/v/t51...,CyD4BLvRppW
4,21,https://instagram.com/p/CyCN8YSSBIj/,377,218947,rotting on the floor hours,False,https://scontent-dfw5-2.cdninstagram.com/v/t51...,CyCN8YSSBIj


In [8]:
df_all_data.rename(columns = {'image_link':'Image URL'},inplace=True)

In [13]:
data_final = df_all_data.merge(df,on = 'Image URL')
ids=['CyOgBu4u6ek','CyLr35yJbc3','CyLj5QBucUt']
data_final = data_final[~data_final['post_id'].isin(ids)]

In [16]:
data_final.head()

,Unnamed: 0,post_link,comment_count,likes_count,caption,is_video,Image URL,post_id,Post URL,Labels/Topics,Safe Search Adult,Safe Search Medical,Safe Search Spoof,Safe Search Violence,Safe Search Racy
3,20,https://instagram.com/p/CyD4BLvRppW/,157,22146,daily affirmation courtesy of milton\n\nNEW EP...,False,https://scontent-dfw5-1.cdninstagram.com/v/t51...,CyD4BLvRppW,https://instagram.com/p/CyD4BLvRppW/,"Forehead, Nose, Chin, Eyebrow, Jaw, Beard, Eye...",VERY_UNLIKELY,UNLIKELY,VERY_LIKELY,UNLIKELY,VERY_UNLIKELY
4,21,https://instagram.com/p/CyCN8YSSBIj/,377,218947,rotting on the floor hours,False,https://scontent-dfw5-2.cdninstagram.com/v/t51...,CyCN8YSSBIj,https://instagram.com/p/CyCN8YSSBIj/,"Water, Sky, Cloud, Shorts, Body of water, Fun,...",UNLIKELY,VERY_UNLIKELY,UNLIKELY,UNLIKELY,POSSIBLE
5,23,https://instagram.com/p/CyBotnouUZp/,391,52657,SEASON 7 OF SELLING SUNSET. NOV 3RD.,False,https://scontent-dfw5-2.cdninstagram.com/v/t51...,CyBotnouUZp,https://instagram.com/p/CyBotnouUZp/,"Water, Gesture, Happy, Sky, Font, Poster, Art,...",UNLIKELY,UNLIKELY,VERY_UNLIKELY,VERY_UNLIKELY,UNLIKELY
6,25,https://instagram.com/p/CyBQrMoPhP7/,638,124887,another british teen show <333333333 EVERYTHIN...,False,https://scontent-dfw5-1.cdninstagram.com/v/t39...,CyBQrMoPhP7,https://instagram.com/p/CyBQrMoPhP7/,"Clothing, Outerwear, Shirt, Photograph, Produc...",UNLIKELY,VERY_UNLIKELY,VERY_LIKELY,UNLIKELY,UNLIKELY
7,29,https://instagram.com/p/Cx-3lcWPJOS/,84,55410,love this journey for her bangs. phoebe dynevo...,False,https://scontent-dfw5-2.cdninstagram.com/v/t51...,Cx-3lcWPJOS,https://instagram.com/p/Cx-3lcWPJOS/,"Forehead, Hair, Face, Lip, Chin, Eyebrow, Hair...",VERY_UNLIKELY,UNLIKELY,LIKELY,UNLIKELY,VERY_UNLIKELY


In [17]:
caption_dict = {}
tags_dict = {}
i=0
for image_url in data_final['Image URL']:

    # Fetch the image from the URL
    response = requests.get(image_url)
    response.raise_for_status()  
    
    image = Image.open(BytesIO(response.content))

    # Call the API
    description_result = computervision_client.describe_image(image_url)
    
    # Store captions and confidence scores in the dictionary
    caption_dict[image_url] = {}
    if len(description_result.captions) == 0:
        caption_dict[image_url]["No description"] = 0.0
    else:
        for caption in description_result.captions:
            caption_dict[image_url][caption.text] = caption.confidence    
    # Call the API
    tags_result = computervision_client.tag_image(image_url)
    
    # Store tags and confidence scores in the dictionary
    tags_dict[image_url] = {}
    if len(tags_result.tags) == 0:
        tags_dict[image_url]["No tags"] = 0.0
    else:
        for tag in tags_result.tags:
            tags_dict[image_url][tag.name] = tag.confidence
    i+=1
    print("Image done:",i)

Image done: 1
Image done: 2
Image done: 3
Image done: 4
Image done: 5
Image done: 6
Image done: 7
Image done: 8
Image done: 9
Image done: 10
Image done: 11
Image done: 12
Image done: 13
Image done: 14
Image done: 15
Image done: 16
Image done: 17
Image done: 18
Image done: 19
Image done: 20
Image done: 21
Image done: 22
Image done: 23
Image done: 24
Image done: 25
Image done: 26
Image done: 27
Image done: 28
Image done: 29
Image done: 30
Image done: 31
Image done: 32
Image done: 33
Image done: 34
Image done: 35
Image done: 36
Image done: 37
Image done: 38
Image done: 39
Image done: 40
Image done: 41
Image done: 42
Image done: 43
Image done: 44
Image done: 45
Image done: 46
Image done: 47
Image done: 48
Image done: 49
Image done: 50
Image done: 51
Image done: 52
Image done: 53
Image done: 54
Image done: 55
Image done: 56
Image done: 57
Image done: 58
Image done: 59
Image done: 60
Image done: 61
Image done: 62
Image done: 63
Image done: 64
Image done: 65
Image done: 66
Image done: 67
Imag

In [18]:
tags_df = pd.DataFrame.from_dict(tags_dict, orient='index')
display(tags_df)

,human face,person,screenshot,text,chin,forehead,clothing,man,eyebrow,jaw,...,jheri curl,ringlet,dreadlocks,thigh,abdomen,stomach,navel,underwear,porch,moon
https://scontent-dfw5-1.cdninstagram.com/v/t51.2885-15/386465525_997729521395449_5560728687591194695_n.jpg?stp=dst-jpg_e35&_nc_ht=scontent-dfw5-1.cdninstagram.com&_nc_cat=101&_nc_ohc=8sTl-d0Qy58AX_ffUph&edm=APU89FABAAAA&ccb=7-5&oh=00_AfCZR0QrPfuQL-UgQdIxcVFVJ0H04mmpF4VoVd3r_4TgGw&oe=652C6DC2&_nc_sid=bc0c2c,0.998515,0.990776,0.971924,0.965324,0.935806,0.933579,0.924988,0.924851,0.91281,0.887809,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
https://scontent-dfw5-1.cdninstagram.com/v/t39.30808-6/386651227_711914570977513_781690418775495798_n.jpg?stp=dst-jpg_e15_fr_s1080x1080&_nc_ht=scontent-dfw5-1.cdninstagram.com&_nc_cat=101&_nc_ohc=WRoEYVbc6ToAX_-yLH3&edm=APU89FAAAAAA&ccb=7-5&oh=00_AfCOxN-U7qSH0V9aLB7l6J2k9X52_k50ZBfJaouDNrgfXw&oe=652BD0F6&_nc_sid=bc0c2c,0.999131,0.991270,NaN,NaN,NaN,NaN,0.997746,0.913560,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
https://scontent-dfw5-2.cdninstagram.com/v/t51.2885-15/385865025_293214653492116_5568725053782254980_n.jpg?stp=dst-jpg_e35&_nc_ht=scontent-dfw5-2.cdninstagram.com&_nc_cat=104&_nc_ohc=Q_rAG1i_pdoAX_2Ipk9&edm=APU89FABAAAA&ccb=7-5&oh=00_AfDLjlBI0s_EQdAIVaS7aHd2D_jHAp5hJOMTZzJQZ8OCbQ&oe=652B3E6D&_nc_sid=bc0c2c,0.996051,0.990912,NaN,NaN,NaN,NaN,0.945397,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
https://scontent-dfw5-2.cdninstagram.com/v/t51.2885-15/386121467_1021281382545141_6933565182577406734_n.jpg?stp=dst-jpg_e35_s1080x1080&_nc_ht=scontent-dfw5-2.cdninstagram.com&_nc_cat=109&_nc_ohc=KnNU_gNqwbIAX_duG0-&edm=APU89FABAAAA&ccb=7-5&oh=00_AfCTPJKVdvQeHzecER8fzViWJdmidm_7MT8jDOGC_g-SfA&oe=652BB986&_nc_sid=bc0c2c,0.997144,0.996984,NaN,NaN,NaN,NaN,0.996150,0.890697,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
https://scontent-dfw5-2.cdninstagram.com/v/t51.2885-15/385407117_213267845101256_2945357874940267779_n.jpg?stp=dst-jpg_e35_p1080x1080&_nc_ht=scontent-dfw5-2.cdninstagram.com&_nc_cat=108&_nc_ohc=HLe4L9wLv-UAX8VslFi&edm=APU89FABAAAA&ccb=7-5&oh=00_AfAt5SNIdoxiigritulCzMIMhgTa55brww6_-YG-5GpnBg&oe=652C9C8F&_nc_sid=bc0c2c,0.997930,0.998106,NaN,NaN,NaN,NaN,0.977260,0.943994,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
https://scontent-dfw5-2.cdninstagram.com/v/t51.2885-15/336019310_1244043036222632_8868390307707901054_n.jpg?stp=dst-jpg_e35&_nc_ht=scontent-dfw5-2.cdninstagram.com&_nc_cat=108&_nc_ohc=Ct3vkmFQTOsAX918LRy&edm=APU89FABAAAA&ccb=7-5&oh=00_AfAKOg_S2Tn4FiOfNKFsjJZJkHI33eRuDfBgEIqjCYhoMA&oe=652BD387&_nc_sid=bc0c2c,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
https://scontent-dfw5-2.cdninstagram.com/v/t39.30808-6/340970206_1150274659235512_1097989480365950204_n.jpg?stp=dst-jpg_e35_p1080x1080_sh0.08&_nc_ht=scontent-dfw5-2.cdninstagram.com&_nc_cat=104&_nc_ohc=RJBH_fm6wjAAX_qelCT&edm=APU89FAAAAAA&ccb=7-5&oh=00_AfDEnitVvsh03A_Ly1i1CiGcG2rjgCd499O612HWs4holQ&oe=652C1A39&_nc_sid=bc0c2c,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
https://scontent-dfw5-1.cdninstagram.com/v/t51.2885-15/344853094_6075116035900985_1628488902754827528_n.jpg?stp=dst-jpg_e35_s1080x1080&_nc_ht=scontent-dfw5-1.cdninstagram.com&_nc_cat=105&_nc_ohc=VpL3VkiX3oUAX-c-Mmc&edm=APU89FABAAAA&ccb=7-5&oh=00_AfCYGhP0MxOvA3Y8ntXtAolTyA0lR01co-OLaA-osDKsOg&oe=652C5E16&_nc_sid=bc0c2c,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
https://scontent-dfw5-2.cdninstagram.com/v/t51.2885-15/358798503_212403408007790_597259153194846194_n.jpg?stp=dst-jpg_e35&_nc_ht=scontent-dfw5-2.cdninstagram.com&_nc_cat=102&_nc_ohc=0KoGmKWj0CIAX_F5AuH&edm=APU89FABAAAA&ccb=7-5&oh=00_AfAdP9TMduZkr-aqTPIy-8w14he4iOHp_W6J79GFsUpuXQ&oe=652D046A&_nc_sid=bc0c2c,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
tags_df.reset_index(level=0, inplace=True)
tags_df.rename(columns={'index':'Image URL'}, inplace=True)
#tags_df.drop(columns='level_0',inplace=True)
tags_df.head()

,Image URL,human face,person,screenshot,text,chin,forehead,clothing,man,eyebrow,...,jheri curl,ringlet,dreadlocks,thigh,abdomen,stomach,navel,underwear,porch,moon
0,https://scontent-dfw5-1.cdninstagram.com/v/t51...,0.998515,0.990776,0.971924,0.965324,0.935806,0.933579,0.924988,0.924851,0.91281,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://scontent-dfw5-1.cdninstagram.com/v/t39...,0.999131,0.991270,NaN,NaN,NaN,NaN,0.997746,0.913560,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://scontent-dfw5-2.cdninstagram.com/v/t51...,0.996051,0.990912,NaN,NaN,NaN,NaN,0.945397,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://scontent-dfw5-2.cdninstagram.com/v/t51...,0.997144,0.996984,NaN,NaN,NaN,NaN,0.996150,0.890697,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://scontent-dfw5-2.cdninstagram.com/v/t51...,0.997930,0.998106,NaN,NaN,NaN,NaN,0.977260,0.943994,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [84]:
df1 = pd.read_csv('Netflix_Image_URLs_Output_vF.csv')
df1.head()

,Unnamed: 0,Post URL,Image URL,Labels/Topics,Safe Search Adult,Safe Search Medical,Safe Search Spoof,Safe Search Violence,Safe Search Racy
0,CyD4BLvRppW,https://instagram.com/p/CyD4BLvRppW/,https://scontent-dfw5-1.cdninstagram.com/v/t51...,"Forehead, Nose, Chin, Eyebrow, Jaw, Beard, Eye...",VERY_UNLIKELY,UNLIKELY,VERY_UNLIKELY,UNLIKELY,VERY_UNLIKELY
1,CyCN8YSSBIj,https://instagram.com/p/CyCN8YSSBIj/,https://scontent-dfw5-2.cdninstagram.com/v/t51...,"Water, Sky, Cloud, Shorts, Body of water, Fun,...",UNLIKELY,VERY_UNLIKELY,LIKELY,UNLIKELY,POSSIBLE
2,CyBotnouUZp,https://instagram.com/p/CyBotnouUZp/,https://scontent-dfw5-2.cdninstagram.com/v/t51...,"Water, Gesture, Happy, Sky, Font, Poster, Art,...",UNLIKELY,UNLIKELY,VERY_LIKELY,VERY_UNLIKELY,UNLIKELY
3,CyBQrMoPhP7,https://instagram.com/p/CyBQrMoPhP7/,https://scontent-dfw5-1.cdninstagram.com/v/t39...,"Clothing, Outerwear, Shirt, Photograph, Produc...",UNLIKELY,VERY_UNLIKELY,VERY_UNLIKELY,UNLIKELY,UNLIKELY
4,Cx-3lcWPJOS,https://instagram.com/p/Cx-3lcWPJOS/,https://scontent-dfw5-2.cdninstagram.com/v/t51...,"Forehead, Hair, Face, Lip, Chin, Eyebrow, Hair...",VERY_UNLIKELY,UNLIKELY,UNLIKELY,UNLIKELY,VERY_UNLIKELY


In [86]:
data_final = df_all_data.merge(df1,on = 'Image URL')

netflix_tags=data_final.merge(tags_df)
netflix_tags

,Unnamed: 0_x,post_link,comment_count,likes_count,caption,is_video,Image URL,post_id,Unnamed: 0_y,Post URL,...,jheri curl,ringlet,dreadlocks,thigh,abdomen,stomach,navel,underwear,porch,moon
0,20,https://instagram.com/p/CyD4BLvRppW/,157,22146,daily affirmation courtesy of milton\n\nNEW EP...,False,https://scontent-dfw5-1.cdninstagram.com/v/t51...,CyD4BLvRppW,CyD4BLvRppW,https://instagram.com/p/CyD4BLvRppW/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,21,https://instagram.com/p/CyCN8YSSBIj/,377,218947,rotting on the floor hours,False,https://scontent-dfw5-2.cdninstagram.com/v/t51...,CyCN8YSSBIj,CyCN8YSSBIj,https://instagram.com/p/CyCN8YSSBIj/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,23,https://instagram.com/p/CyBotnouUZp/,391,52657,SEASON 7 OF SELLING SUNSET. NOV 3RD.,False,https://scontent-dfw5-2.cdninstagram.com/v/t51...,CyBotnouUZp,CyBotnouUZp,https://instagram.com/p/CyBotnouUZp/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,25,https://instagram.com/p/CyBQrMoPhP7/,638,124887,another british teen show <333333333 EVERYTHIN...,False,https://scontent-dfw5-1.cdninstagram.com/v/t39...,CyBQrMoPhP7,CyBQrMoPhP7,https://instagram.com/p/CyBQrMoPhP7/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,29,https://instagram.com/p/Cx-3lcWPJOS/,84,55410,love this journey for her bangs. phoebe dynevo...,False,https://scontent-dfw5-2.cdninstagram.com/v/t51...,Cx-3lcWPJOS,Cx-3lcWPJOS,https://instagram.com/p/Cx-3lcWPJOS/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
256,985,https://instagram.com/p/CpTJYK1td6F/,709,303206,these SEX/LIFE photos are a gift from me to you 😅,False,https://scontent-dfw5-1.cdninstagram.com/v/t39...,CpTJYK1td6F,CpTJYK1td6F,https://instagram.com/p/CpTJYK1td6F/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
257,988,https://instagram.com/p/CpS01sLpiMB/,147,72395,obsessed 😍😍,False,https://scontent-dfw5-1.cdninstagram.com/v/t51...,CpS01sLpiMB,CpS01sLpiMB,https://instagram.com/p/CpS01sLpiMB/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
258,991,https://instagram.com/p/CpQkmn7uO1r/,297,50692,points were made.,False,https://scontent-dfw5-2.cdninstagram.com/v/t51...,CpQkmn7uO1r,CpQkmn7uO1r,https://instagram.com/p/CpQkmn7uO1r/,...,0.88703,0.881622,0.855463,NaN,NaN,NaN,NaN,NaN,NaN,NaN
259,992,https://instagram.com/p/CpQP_13JHoV/,739,633396,the pogues at the chateau are my forever mood ...,False,https://scontent-dfw5-2.cdninstagram.com/v/t51...,CpQP_13JHoV,CpQP_13JHoV,https://instagram.com/p/CpQP_13JHoV/,...,NaN,NaN,NaN,0.958456,0.9445,0.914622,0.898263,0.685672,0.646265,NaN


In [34]:
column_names=netflix_tags.columns
display(column_names[:30])

Index(['Unnamed: 0', 'post_link', 'comment_count', 'likes_count', 'caption',
       'is_video', 'Image URL', 'post_id', 'Post URL', 'Labels/Topics',
       'Safe Search Adult', 'Safe Search Medical', 'Safe Search Spoof',
       'Safe Search Violence', 'Safe Search Racy', 'human face', 'person',
       'screenshot', 'text', 'chin', 'forehead', 'clothing', 'man', 'eyebrow',
       'jaw', 'skin', 'cheek', 'human', 'outdoor', 'sky'],
      dtype='object')

In [87]:
netflix_tags['Spoof'] = netflix_tags['Safe Search Spoof'].apply(lambda x: 'Spoof' if x in ['VERY LIKELY', 'LIKELY', 'POSSIBLE'] else 'Non-Spoof')

In [88]:
netflix_tags['poster_post'] = netflix_tags['poster'].apply(lambda x: 'Poster' if x > 0 else 'Non-Poster')

In [89]:
# Assuming netflix_tags is your DataFrame
def classify_poster(row):
    if row['poster_post'] == 'Poster':
        return 'Poster'
    elif row['poster_post'] == 'Non-Poster':
        if row['Spoof'] == 'Spoof':
            return 'Meme'
        else:
            return 'Others'
    else:
        return 'Unknown'  # Optional, for handling unexpected values

netflix_tags['post_type'] = netflix_tags.apply(classify_poster, axis=1)

In [90]:
netflix_tags.head(5)

,Unnamed: 0_x,post_link,comment_count,likes_count,caption,is_video,Image URL,post_id,Unnamed: 0_y,Post URL,...,thigh,abdomen,stomach,navel,underwear,porch,moon,Spoof,poster_post,post_type
0,20,https://instagram.com/p/CyD4BLvRppW/,157,22146,daily affirmation courtesy of milton\n\nNEW EP...,False,https://scontent-dfw5-1.cdninstagram.com/v/t51...,CyD4BLvRppW,CyD4BLvRppW,https://instagram.com/p/CyD4BLvRppW/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Non-Spoof,Non-Poster,Others
1,21,https://instagram.com/p/CyCN8YSSBIj/,377,218947,rotting on the floor hours,False,https://scontent-dfw5-2.cdninstagram.com/v/t51...,CyCN8YSSBIj,CyCN8YSSBIj,https://instagram.com/p/CyCN8YSSBIj/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Spoof,Non-Poster,Meme
2,23,https://instagram.com/p/CyBotnouUZp/,391,52657,SEASON 7 OF SELLING SUNSET. NOV 3RD.,False,https://scontent-dfw5-2.cdninstagram.com/v/t51...,CyBotnouUZp,CyBotnouUZp,https://instagram.com/p/CyBotnouUZp/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Non-Spoof,Poster,Poster
3,25,https://instagram.com/p/CyBQrMoPhP7/,638,124887,another british teen show <333333333 EVERYTHIN...,False,https://scontent-dfw5-1.cdninstagram.com/v/t39...,CyBQrMoPhP7,CyBQrMoPhP7,https://instagram.com/p/CyBQrMoPhP7/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Non-Spoof,Non-Poster,Others
4,29,https://instagram.com/p/Cx-3lcWPJOS/,84,55410,love this journey for her bangs. phoebe dynevo...,False,https://scontent-dfw5-2.cdninstagram.com/v/t51...,Cx-3lcWPJOS,Cx-3lcWPJOS,https://instagram.com/p/Cx-3lcWPJOS/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Non-Spoof,Non-Poster,Others


In [91]:
netflix_scores = pd.read_csv('netflix_merged_df.csv')
netflix_scores.head(5)

,Unnamed: 0,post_link,post_id,comment_count,likes_count,caption,is_video,image_link,normalized_likes,sentiment_score,compound_score
0,750,https://instagram.com/p/CruFXeJvhd2/,CruFXeJvhd2,1057,979290,WEDNESDAY SLAYING ON A MONDAY AT THE MET GALA\...,False,https://scontent-dfw5-2.cdninstagram.com/v/t51...,1.0000,0.5423,0.5423
1,519,https://instagram.com/p/CtmwukAhtxH/,CtmwukAhtxH,2874,874007,Your first look at Penelope and Colin in Bridg...,False,https://scontent-dfw5-2.cdninstagram.com/v/t39...,0.8900,0.5718,0.5089
2,585,https://instagram.com/p/CtKQh49O84M/,CtKQh49O84M,1538,845184,can't stop staring at these jenna ortega photo...,False,https://scontent-dfw5-2.cdninstagram.com/v/t51...,0.8599,0.4856,0.4176
3,620,https://instagram.com/p/Cs1i9ZFBlOt/,Cs1i9ZFBlOt,2051,740088,that feeling when you’re laying besides your c...,False,https://scontent-dfw5-2.cdninstagram.com/v/t51...,0.7501,0.4940,0.3705
4,984,https://instagram.com/p/CpTQQTdtyBd/,CpTQQTdtyBd,2493,706117,THEM!!!!!,False,https://scontent-dfw5-2.cdninstagram.com/v/t39...,0.7146,0.5768,0.4122


In [92]:
netflix_final_df=netflix_scores.merge(netflix_tags,on='post_id')
netflix_final_df

,Unnamed: 0,post_link_x,post_id,comment_count_x,likes_count_x,caption_x,is_video_x,image_link,normalized_likes,sentiment_score,...,thigh,abdomen,stomach,navel,underwear,porch,moon,Spoof,poster_post,post_type
0,750,https://instagram.com/p/CruFXeJvhd2/,CruFXeJvhd2,1057,979290,WEDNESDAY SLAYING ON A MONDAY AT THE MET GALA\...,False,https://scontent-dfw5-2.cdninstagram.com/v/t51...,1.0000,0.5423,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Non-Spoof,Non-Poster,Others
1,519,https://instagram.com/p/CtmwukAhtxH/,CtmwukAhtxH,2874,874007,Your first look at Penelope and Colin in Bridg...,False,https://scontent-dfw5-2.cdninstagram.com/v/t39...,0.8900,0.5718,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Non-Spoof,Non-Poster,Others
2,585,https://instagram.com/p/CtKQh49O84M/,CtKQh49O84M,1538,845184,can't stop staring at these jenna ortega photo...,False,https://scontent-dfw5-2.cdninstagram.com/v/t51...,0.8599,0.4856,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Non-Spoof,Non-Poster,Others
3,620,https://instagram.com/p/Cs1i9ZFBlOt/,Cs1i9ZFBlOt,2051,740088,that feeling when you’re laying besides your c...,False,https://scontent-dfw5-2.cdninstagram.com/v/t51...,0.7501,0.4940,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Non-Spoof,Non-Poster,Others
4,984,https://instagram.com/p/CpTQQTdtyBd/,CpTQQTdtyBd,2493,706117,THEM!!!!!,False,https://scontent-dfw5-2.cdninstagram.com/v/t39...,0.7146,0.5768,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Spoof,Non-Poster,Meme
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
252,555,https://instagram.com/p/Ctb_NbjN8u2/,Ctb_NbjN8u2,139,28064,Quarterback premieres July 12 but huddle up: t...,False,https://scontent-dfw5-2.cdninstagram.com/v/t39...,0.0062,0.5120,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Spoof,Poster,Poster
253,97,https://instagram.com/p/CxVzwX1xeyq/,CxVzwX1xeyq,60,27541,all v real all v relatable\nSPY KIDS: ARMAGEDD...,False,https://scontent-dfw5-1.cdninstagram.com/v/t51...,0.0056,0.4955,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Spoof,Non-Poster,Meme
254,394,https://instagram.com/p/Cum0l5LuPrV/,Cum0l5LuPrV,107,26479,we said beware cheap imitations. THEY CLONED T...,False,https://scontent-dfw5-2.cdninstagram.com/v/t51...,0.0045,0.5460,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Spoof,Poster,Poster
255,335,https://instagram.com/p/CvKqBi_uZ7y/,CvKqBi_uZ7y,71,25569,Women in STEM!,False,https://scontent-dfw5-2.cdninstagram.com/v/t51...,0.0036,0.5508,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Non-Spoof,Non-Poster,Others


In [59]:
netflix_final_df.to_csv('netflix_final_data.csv', index=False)

In [53]:
netflix_final_df['compound_score']

0      0.5423
1      0.5089
2      0.4176
3      0.3705
4      0.4122
        ...  
252    0.0032
253    0.0028
254    0.0025
255    0.0020
256    0.0000
Name: compound_score, Length: 257, dtype: float64

### Final Values for the different types of posts in Netflix

In [93]:
post_type_scores=pd.DataFrame({'score_avg':netflix_final_df.groupby('post_type')['compound_score'].mean(),'sample':netflix_final_df.groupby('post_type')['compound_score'].size()})
display(round(post_type_scores,2))

,score_avg,sample
post_type,,
Meme,0.08,65
Others,0.09,166
Poster,0.09,26


Disney

In [63]:
disney_df = pd.read_csv('DisneyPlus_Image_URLs_Output_vF.csv')
disney_df.head()

,Post URL,Image URL,Labels/Topics,Safe Search Adult,Safe Search Medical,Safe Search Spoof,Safe Search Violence,Safe Search Racy
0,https://instagram.com/p/CyEfFWRPp-Y/,https://scontent-dfw5-2.cdninstagram.com/v/t51...,"Cartoon, Font, Art, Happy, Event, Photo captio...",VERY_UNLIKELY,VERY_UNLIKELY,VERY_UNLIKELY,UNLIKELY,POSSIBLE
1,https://instagram.com/p/CyEKIYtxEGl/,https://scontent-dfw5-2.cdninstagram.com/v/t51...,"Light, Nature, Lighting, Organism, Art, Font, ...",VERY_UNLIKELY,VERY_UNLIKELY,VERY_UNLIKELY,VERY_UNLIKELY,VERY_UNLIKELY
2,https://instagram.com/p/Cx-6CDzLux7/,https://scontent-dfw5-1.cdninstagram.com/v/t51...,"Poster, Cg artwork, Flash photography, Font, D...",VERY_UNLIKELY,VERY_UNLIKELY,VERY_UNLIKELY,UNLIKELY,VERY_UNLIKELY
3,https://instagram.com/p/Cx525L9xJbw/,https://scontent-dfw5-2.cdninstagram.com/v/t51...,"Flash photography, Eyewear, Font, Technology, ...",VERY_UNLIKELY,VERY_UNLIKELY,UNLIKELY,VERY_UNLIKELY,VERY_UNLIKELY
4,https://instagram.com/p/Cx3EXD_hxmw/,https://scontent-dfw5-2.cdninstagram.com/v/t39...,"World, Blue, Azure, Organism, Font, Electric b...",VERY_UNLIKELY,VERY_UNLIKELY,VERY_UNLIKELY,VERY_UNLIKELY,VERY_UNLIKELY


In [64]:
disney_all_data = pd.read_csv('DisneyPlus_All_Input_vF.csv')
disney_all_data['post_id'] = df_all_data['post_link'].str.rsplit('/', 2).str[-2]
disney_all_data.head()

C:\Users\vaish\AppData\Local\Temp\ipykernel_61528\919945820.py:2: FutureWarning: In a future version of pandas all arguments of StringMethods.rsplit except for the argument 'pat' will be keyword-only.
  disney_all_data['post_id'] = df_all_data['post_link'].str.rsplit('/', 2).str[-2]


,Unnamed: 0,post_link,post_id,comment_count,likes_count,caption,is_video,image_link
0,7,https://instagram.com/p/CyEfFWRPp-Y/,CyOgBu4u6ek,147,37853,Cat's outta the bag! These newly restored Walt...,False,https://scontent-dfw5-2.cdninstagram.com/v/t51...
1,8,https://instagram.com/p/CyEKIYtxEGl/,CyLr35yJbc3,119,46864,Make broom for this original art honoring 30 y...,False,https://scontent-dfw5-2.cdninstagram.com/v/t51...
2,10,https://instagram.com/p/Cx-6CDzLux7/,CyLj5QBucUt,183,40272,There's always room for one more.\n\nHaunted M...,False,https://scontent-dfw5-1.cdninstagram.com/v/t51...
3,14,https://instagram.com/p/Cx525L9xJbw/,CyD4BLvRppW,3264,1261650,Time to mark the calendar. 🗓️\n\nMarvel Studio...,False,https://scontent-dfw5-2.cdninstagram.com/v/t51...
4,17,https://instagram.com/p/Cx3EXD_hxmw/,CyCN8YSSBIj,73,17114,Be our 👻ghost👻 and stream to your h̶̶̶e̶̶̶a̶̶̶...,False,https://scontent-dfw5-2.cdninstagram.com/v/t39...


In [65]:
disney_all_data.rename(columns = {'image_link':'Image URL'},inplace=True)

In [66]:
caption_dict_disney = {}
tags_dict_disney = {}
i=0
for image_url in disney_all_data['Image URL']:

    # Fetch the image from the URL
    response = requests.get(image_url)
    response.raise_for_status()  
    
    image = Image.open(BytesIO(response.content))

    # Call the API
    description_result = computervision_client.describe_image(image_url)
    
    # Store captions and confidence scores in the dictionary
    caption_dict_disney[image_url] = {}
    if len(description_result.captions) == 0:
        caption_dict_disney[image_url]["No description"] = 0.0
    else:
        for caption in description_result.captions:
            caption_dict_disney[image_url][caption.text] = caption.confidence    
    # Call the API
    tags_result = computervision_client.tag_image(image_url)
    
    # Store tags and confidence scores in the dictionary
    tags_dict_disney[image_url] = {}
    if len(tags_result.tags) == 0:
        tags_dict_disney[image_url]["No tags"] = 0.0
    else:
        for tag in tags_result.tags:
            tags_dict_disney[image_url][tag.name] = tag.confidence
    i+=1
    print("Image done:",i)

Image done: 1
Image done: 2
Image done: 3
Image done: 4
Image done: 5
Image done: 6
Image done: 7
Image done: 8
Image done: 9
Image done: 10
Image done: 11
Image done: 12
Image done: 13
Image done: 14
Image done: 15
Image done: 16
Image done: 17
Image done: 18
Image done: 19
Image done: 20
Image done: 21
Image done: 22
Image done: 23
Image done: 24
Image done: 25
Image done: 26
Image done: 27
Image done: 28
Image done: 29
Image done: 30
Image done: 31
Image done: 32
Image done: 33
Image done: 34
Image done: 35
Image done: 36
Image done: 37
Image done: 38
Image done: 39
Image done: 40
Image done: 41
Image done: 42
Image done: 43
Image done: 44
Image done: 45
Image done: 46
Image done: 47
Image done: 48
Image done: 49
Image done: 50
Image done: 51
Image done: 52
Image done: 53
Image done: 54
Image done: 55
Image done: 56
Image done: 57
Image done: 58
Image done: 59
Image done: 60
Image done: 61
Image done: 62
Image done: 63
Image done: 64
Image done: 65
Image done: 66
Image done: 67
Imag

ComputerVisionErrorResponseException: (429) Requests to the Tag Image Operation under Computer Vision API (v3.2) have exceeded call rate limit of your current ComputerVision F0 pricing tier. Please retry after 1 second. To increase your rate limit switch to a paid tier.

In [67]:
tags_df_disney = pd.DataFrame.from_dict(tags_dict_disney, orient='index')
display(tags_df_disney)

,text,poster,cartoon,graphic design,fiction,book,art,illustration,halloween,pumpkin,...,bridal veil,crown,jewellery,headpiece,tennis,athletic game,racket,lemur,audience,crowd
https://scontent-dfw5-2.cdninstagram.com/v/t51.2885-15/386361737_1741580889642417_8430299496059101760_n.jpg?stp=dst-jpg_e35_p1080x1080&_nc_ht=scontent-dfw5-2.cdninstagram.com&_nc_cat=109&_nc_ohc=VEXzOQAgQZAAX-JxBSx&edm=AOQ1c0wBAAAA&ccb=7-5&ig_cache_key=MzIwNzgyNTUzNTQ2Mzc5Mzg2OQ%3D%3D.2-ccb7-5&oh=00_AfC4JAJ3CGCmO9y01yPCEBLaKmhtFBSt0cY_leID9NVADg&oe=652C09BC&_nc_sid=8b3546,0.999666,0.959923,0.922566,0.889302,0.886023,0.848518,0.468368,0.446872,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
https://scontent-dfw5-2.cdninstagram.com/v/t51.2885-15/386465644_292123236967256_8030468979791487055_n.jpg?stp=dst-jpg_e35_p1080x1080&_nc_ht=scontent-dfw5-2.cdninstagram.com&_nc_cat=108&_nc_ohc=1AZxsxtmceIAX__EiRy&edm=AOQ1c0wBAAAA&ccb=7-5&ig_cache_key=MzIwNzczMzM5MTM1MzIwMTA2MQ%3D%3D.2-ccb7-5&oh=00_AfBTX0JQqLBdDGgJBHWnPiwYQ4kGe9Cj-kp8CrHwM_lBtg&oe=652CFAEF&_nc_sid=8b3546,0.939153,NaN,0.899996,NaN,NaN,NaN,NaN,NaN,0.918431,0.860741,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
https://scontent-dfw5-1.cdninstagram.com/v/t51.2885-15/386307838_280673198144511_665367217931045365_n.jpg?stp=dst-jpg_e35_p1080x1080&_nc_ht=scontent-dfw5-1.cdninstagram.com&_nc_cat=103&_nc_ohc=QVR-yDs6i6EAX-4zwRl&edm=AOQ1c0wBAAAA&ccb=7-5&ig_cache_key=MzIwNjI1NTIxMjk1MDkwODAyNw%3D%3D.2-ccb7-5&oh=00_AfAYHL2BYQ7ddJA9igeAkIKTy2X9LKHkMKq9KVk4748jhQ&oe=652C6487&_nc_sid=8b3546,0.998839,0.918493,NaN,NaN,NaN,0.911752,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
https://scontent-dfw5-2.cdninstagram.com/v/t51.2885-15/385049931_824508265881396_7206048436675947825_n.jpg?stp=dst-jpg_e35&_nc_ht=scontent-dfw5-2.cdninstagram.com&_nc_cat=100&_nc_ohc=bv6S4vckDN8AX_XQDA1&edm=APU89FABAAAA&ccb=7-5&oh=00_AfASY_HPjYXZAPgZs3tMJuSXcft5VKTOMfiTF5udCl9Ivg&oe=652CE841&_nc_sid=bc0c2c,0.998541,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
https://scontent-dfw5-2.cdninstagram.com/v/t39.30808-6/384745744_323924100291453_3970620487030742196_n.jpg?stp=dst-jpg_e15_fr_p1080x1080&_nc_ht=scontent-dfw5-2.cdninstagram.com&_nc_cat=109&_nc_ohc=-AvfNW0bObEAX80jSj4&edm=APU89FAAAAAA&ccb=7-5&oh=00_AfATinQVXK92PWNpAFhfw80nrooTrfPJDZYBrQKR8x0Mew&oe=652B6C67&_nc_sid=bc0c2c,0.999626,0.874778,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
https://scontent-dfw5-1.cdninstagram.com/v/t51.2885-15/335930860_580180034162122_7931738119005902892_n.jpg?stp=dst-jpg_e35_p1080x1080&_nc_ht=scontent-dfw5-1.cdninstagram.com&_nc_cat=110&_nc_ohc=NAKGYfrpmOIAX8jCuH_&edm=APU89FABAAAA&ccb=7-5&oh=00_AfCD-EkxulDWY72pZSiIZ6mWT86xqklfprQ3_1-qvPNHNQ&oe=652B52BB&_nc_sid=bc0c2c,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.850278,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
https://scontent-dfw5-1.cdninstagram.com/v/t51.2885-15/377298556_867929068231673_4500068279522305634_n.jpg?stp=dst-jpg_e35_s1080x1080&_nc_ht=scontent-dfw5-1.cdninstagram.com&_nc_cat=105&_nc_ohc=dsc79iV4874AX91_q8e&edm=APU89FABAAAA&ccb=7-5&oh=00_AfBWG7hCO_hJ8O50AjHbOQQms61xN74fGpRKha_SSoFtVw&oe=652CFC8F&_nc_sid=bc0c2c,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
https://scontent-dfw5-1.cdninstagram.com/v/t51.2885-15/363507733_110695228778338_6134744925454366042_n.jpg?stp=dst-jpg_e35_p1080x1080&_nc_ht=scontent-dfw5-1.cdninstagram.com&_nc_cat=110&_nc_ohc=Jxkawe29vcgAX9RTS4v&edm=APU89FABAAAA&ccb=7-5&oh=00_AfD6uYpqZpJYAiachwswdzeT160CazOjHW4-5TtHsru07g&oe=652C4CC6&_nc_sid=bc0c2c,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
https://scontent-dfw5-1.cdninstagram.com/v/t51.2885-15/342333233_1589203701562115_1422835592592408486_n.jpg?stp=dst-jpg_e35_p1080x1080&_nc_ht=scontent-dfw5-1.cdninstagram.com&_nc_cat=111&_nc_ohc=edqUoj3lxo0AX9rqmMv&edm=APU89FABAAAA&ccb=7-5&oh=00_AfDjYVKqN7_CT4n

In [68]:
tags_df_disney.reset_index(level=0, inplace=True)
tags_df_disney.rename(columns={'index':'Image URL'}, inplace=True)
#tags_df.drop(columns='level_0',inplace=True)
tags_df_disney.head()

,Image URL,text,poster,cartoon,graphic design,fiction,book,art,illustration,halloween,...,bridal veil,crown,jewellery,headpiece,tennis,athletic game,racket,lemur,audience,crowd
0,https://scontent-dfw5-2.cdninstagram.com/v/t51...,0.999666,0.959923,0.922566,0.889302,0.886023,0.848518,0.468368,0.446872,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://scontent-dfw5-2.cdninstagram.com/v/t51...,0.939153,NaN,0.899996,NaN,NaN,NaN,NaN,NaN,0.918431,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://scontent-dfw5-1.cdninstagram.com/v/t51...,0.998839,0.918493,NaN,NaN,NaN,0.911752,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://scontent-dfw5-2.cdninstagram.com/v/t51...,0.998541,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://scontent-dfw5-2.cdninstagram.com/v/t39...,0.999626,0.874778,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [71]:
disney_tags=disney_df.merge(tags_df_disney,on='Image URL')
disney_tags

,Post URL,Image URL,Labels/Topics,Safe Search Adult,Safe Search Medical,Safe Search Spoof,Safe Search Violence,Safe Search Racy,text,poster,...,bridal veil,crown,jewellery,headpiece,tennis,athletic game,racket,lemur,audience,crowd
0,https://instagram.com/p/CyEfFWRPp-Y/,https://scontent-dfw5-2.cdninstagram.com/v/t51...,"Cartoon, Font, Art, Happy, Event, Photo captio...",VERY_UNLIKELY,VERY_UNLIKELY,VERY_UNLIKELY,UNLIKELY,POSSIBLE,0.999666,0.959923,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://instagram.com/p/CyEKIYtxEGl/,https://scontent-dfw5-2.cdninstagram.com/v/t51...,"Light, Nature, Lighting, Organism, Art, Font, ...",VERY_UNLIKELY,VERY_UNLIKELY,VERY_UNLIKELY,VERY_UNLIKELY,VERY_UNLIKELY,0.939153,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://instagram.com/p/Cx-6CDzLux7/,https://scontent-dfw5-1.cdninstagram.com/v/t51...,"Poster, Cg artwork, Flash photography, Font, D...",VERY_UNLIKELY,VERY_UNLIKELY,VERY_UNLIKELY,UNLIKELY,VERY_UNLIKELY,0.998839,0.918493,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://instagram.com/p/Cx525L9xJbw/,https://scontent-dfw5-2.cdninstagram.com/v/t51...,"Flash photography, Eyewear, Font, Technology, ...",VERY_UNLIKELY,VERY_UNLIKELY,UNLIKELY,VERY_UNLIKELY,VERY_UNLIKELY,0.998541,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://instagram.com/p/Cx3EXD_hxmw/,https://scontent-dfw5-2.cdninstagram.com/v/t39...,"World, Blue, Azure, Organism, Font, Electric b...",VERY_UNLIKELY,VERY_UNLIKELY,VERY_UNLIKELY,VERY_UNLIKELY,VERY_UNLIKELY,0.999626,0.874778,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,https://instagram.com/p/CpO4DVVsgP_/,https://scontent-dfw5-2.cdninstagram.com/v/t51...,"Photograph, White, Organism, Flash photography...",UNLIKELY,UNLIKELY,POSSIBLE,UNLIKELY,UNLIKELY,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
248,https://instagram.com/p/CpObuTpBXQg/,https://scontent-dfw5-2.cdninstagram.com/v/t51...,"Product, Human, Font, Line, Adaptation, Screen...",UNLIKELY,UNLIKELY,UNLIKELY,UNLIKELY,UNLIKELY,0.991849,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
249,https://instagram.com/p/CpNyEc2Mcwx/,https://scontent-dfw5-2.cdninstagram.com/v/t51...,"Helmet, Vertebrate, World, Organism, Sports ge...",VERY_UNLIKELY,VERY_UNLIKELY,UNLIKELY,UNLIKELY,VERY_UNLIKELY,0.996481,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
250,https://instagram.com/p/CpNs0izJdUG/,https://scontent-dfw5-2.cdninstagram.com/v/t51...,"Human, World, Happy, Poster, Action film, Cg a...",VERY_UNLIKELY,VERY_UNLIKELY,VERY_UNLIKELY,UNLIKELY,VERY_UNLIKELY,0.995532,0.927625,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [72]:
disney_tags['Spoof'] = disney_tags['Safe Search Spoof'].apply(lambda x: 'Spoof' if x in ['VERY LIKELY', 'LIKELY', 'POSSIBLE'] else 'Non-Spoof')

In [73]:
disney_tags['poster_post'] = disney_tags['poster'].apply(lambda x: 'Poster' if x > 0 else 'Non-Poster')

In [75]:
# Assuming netflix_tags is your DataFrame
def classify_poster(row):
    if row['poster_post'] == 'Poster':
        return 'Poster'
    elif row['poster_post'] == 'Non-Poster':
        if row['Spoof'] == 'Spoof':
            return 'Meme'
        else:
            return 'Others'
    else:
        return 'Unknown'  # Optional, for handling unexpected values

disney_tags['post_type'] = disney_tags.apply(classify_poster, axis=1)
disney_tags.head()

,Post URL,Image URL,Labels/Topics,Safe Search Adult,Safe Search Medical,Safe Search Spoof,Safe Search Violence,Safe Search Racy,text,poster,...,headpiece,tennis,athletic game,racket,lemur,audience,crowd,Spoof,poster_post,post_type
0,https://instagram.com/p/CyEfFWRPp-Y/,https://scontent-dfw5-2.cdninstagram.com/v/t51...,"Cartoon, Font, Art, Happy, Event, Photo captio...",VERY_UNLIKELY,VERY_UNLIKELY,VERY_UNLIKELY,UNLIKELY,POSSIBLE,0.999666,0.959923,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Non-Spoof,Poster,Poster
1,https://instagram.com/p/CyEKIYtxEGl/,https://scontent-dfw5-2.cdninstagram.com/v/t51...,"Light, Nature, Lighting, Organism, Art, Font, ...",VERY_UNLIKELY,VERY_UNLIKELY,VERY_UNLIKELY,VERY_UNLIKELY,VERY_UNLIKELY,0.939153,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Non-Spoof,Non-Poster,Others
2,https://instagram.com/p/Cx-6CDzLux7/,https://scontent-dfw5-1.cdninstagram.com/v/t51...,"Poster, Cg artwork, Flash photography, Font, D...",VERY_UNLIKELY,VERY_UNLIKELY,VERY_UNLIKELY,UNLIKELY,VERY_UNLIKELY,0.998839,0.918493,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Non-Spoof,Poster,Poster
3,https://instagram.com/p/Cx525L9xJbw/,https://scontent-dfw5-2.cdninstagram.com/v/t51...,"Flash photography, Eyewear, Font, Technology, ...",VERY_UNLIKELY,VERY_UNLIKELY,UNLIKELY,VERY_UNLIKELY,VERY_UNLIKELY,0.998541,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Non-Spoof,Non-Poster,Others
4,https://instagram.com/p/Cx3EXD_hxmw/,https://scontent-dfw5-2.cdninstagram.com/v/t39...,"World, Blue, Azure, Organism, Font, Electric b...",VERY_UNLIKELY,VERY_UNLIKELY,VERY_UNLIKELY,VERY_UNLIKELY,VERY_UNLIKELY,0.999626,0.874778,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Non-Spoof,Poster,Poster


In [76]:
disney_scores = pd.read_csv('disney_merged_df.csv')
disney_scores.head(5)

,Unnamed: 0,post_link,post_id,comment_count,likes_count,caption,is_video,image_link,normalized_likes,sentiment_score,compound_score
0,298,https://instagram.com/p/Csg0imTRWEZ/,Csg0imTRWEZ,1154,763310,⚠️ Identities not confirmed ⚠️\n\nCheck out th...,False,https://scontent-dfw5-1.cdninstagram.com/v/t51...,1.0000,0.5837,0.5837
1,430,https://instagram.com/p/CrQ46EALMxZ/,CrQ46EALMxZ,4269,734710,Our Spidey-Senses are tingling. 🕷💢\n\nThese #S...,False,https://scontent-dfw5-2.cdninstagram.com/v/t51...,0.9623,0.4827,0.4645
2,153,https://instagram.com/p/Curz5ANrReB/,Curz5ANrReB,3206,690932,Heroes are made in #PercyJackson and the Olymp...,False,https://scontent-dfw5-1.cdninstagram.com/v/t51...,0.9046,0.5845,0.5287
3,124,https://instagram.com/p/CvcrpgPrLk4/,CvcrpgPrLk4,1666,685735,Marvel Studios’ Guardians of the Galaxy Vol. 3...,False,https://scontent-dfw5-1.cdninstagram.com/v/t51...,0.8977,0.6004,0.5390
4,114,https://instagram.com/p/CvnGQvqxEbx/,CvnGQvqxEbx,1547,608011,Groot returns in one month for a treemendous f...,False,https://scontent-dfw5-2.cdninstagram.com/v/t51...,0.7953,0.5193,0.4130


In [81]:
disney_tags['post_id'] = disney_tags['Post URL'].str.rsplit('/', 2).str[-2]
disney_final_df=disney_scores.merge(disney_tags,on='post_id')
disney_final_df

C:\Users\vaish\AppData\Local\Temp\ipykernel_61528\2671263433.py:1: FutureWarning: In a future version of pandas all arguments of StringMethods.rsplit except for the argument 'pat' will be keyword-only.
  disney_tags['post_id'] = disney_tags['Post URL'].str.rsplit('/', 2).str[-2]


,Unnamed: 0,post_link,post_id,comment_count,likes_count,caption,is_video,image_link,normalized_likes,sentiment_score,...,headpiece,tennis,athletic game,racket,lemur,audience,crowd,Spoof,poster_post,post_type
0,298,https://instagram.com/p/Csg0imTRWEZ/,Csg0imTRWEZ,1154,763310,⚠️ Identities not confirmed ⚠️\n\nCheck out th...,False,https://scontent-dfw5-1.cdninstagram.com/v/t51...,1.0000,0.5837,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Non-Spoof,Poster,Poster
1,430,https://instagram.com/p/CrQ46EALMxZ/,CrQ46EALMxZ,4269,734710,Our Spidey-Senses are tingling. 🕷💢\n\nThese #S...,False,https://scontent-dfw5-2.cdninstagram.com/v/t51...,0.9623,0.4827,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Non-Spoof,Non-Poster,Others
2,153,https://instagram.com/p/Curz5ANrReB/,Curz5ANrReB,3206,690932,Heroes are made in #PercyJackson and the Olymp...,False,https://scontent-dfw5-1.cdninstagram.com/v/t51...,0.9046,0.5845,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Non-Spoof,Poster,Poster
3,124,https://instagram.com/p/CvcrpgPrLk4/,CvcrpgPrLk4,1666,685735,Marvel Studios’ Guardians of the Galaxy Vol. 3...,False,https://scontent-dfw5-1.cdninstagram.com/v/t51...,0.8977,0.6004,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Non-Spoof,Poster,Poster
4,114,https://instagram.com/p/CvnGQvqxEbx/,CvnGQvqxEbx,1547,608011,Groot returns in one month for a treemendous f...,False,https://scontent-dfw5-2.cdninstagram.com/v/t51...,0.7953,0.5193,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Non-Spoof,Poster,Poster
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
241,63,https://instagram.com/p/CxGQxqkLo7s/,CxGQxqkLo7s,79,7234,"Twelve unique voices, six original stories, on...",False,https://scontent-dfw5-1.cdninstagram.com/v/t51...,0.0033,0.5666,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Spoof,Poster,Poster
242,308,https://instagram.com/p/CsZTwkXtePy/,CsZTwkXtePy,19,7129,That’s definitely @DefinitelyNotBenWang.\n\nCa...,False,https://scontent-dfw5-2.cdninstagram.com/v/t39...,0.0032,0.5269,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Non-Spoof,Non-Poster,Others
243,289,https://instagram.com/p/CstzOYnhhgL/,CstzOYnhhgL,23,6392,Discover @Kristine_Tompkins‘ life story from a...,False,https://scontent-dfw5-2.cdninstagram.com/v/t39...,0.0022,0.5381,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Spoof,Poster,Poster
244,244,https://instagram.com/p/CtSPsS-BhLg/,CtSPsS-BhLg,14,5019,See travel journalist Jeff Jenkins live his be...,False,https://scontent-dfw5-2.cdninstagram.com/v/t39...,0.0004,0.3147,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Non-Spoof,Poster,Poster


### Final Values for the different types of posts in Disney+

In [82]:
disney_post_type_scores=pd.DataFrame({'score_avg':disney_final_df.groupby('post_type')['compound_score'].mean(),'sample':disney_final_df.groupby('post_type')['compound_score'].size()})
display(round(disney_post_type_scores,2))

,score_avg,sample
post_type,,
Meme,0.02,19
Others,0.04,122
Poster,0.06,105


### Topic Modeling


In [ ]:
raw_data = pd.read_csv(r"C:\Users\rsk29\Documents\UT\Fall\AUD\Project\netflix_Image_tags.csv")
`
raw_data.head()

# Define helper functions
def get_top_n_words(n_top_words, count_vectorizer, text_data):
    '''
    returns a tuple of the top n words in a sample and their 
    accompanying counts, given a CountVectorizer object and text sample
    '''
    vectorized_headlines = count_vectorizer.fit_transform(text_data.values)
    vectorized_total = np.sum(vectorized_headlines, axis=0)
    word_indices = np.flip(np.argsort(vectorized_total)[0,:], 1)
    word_values = np.flip(np.sort(vectorized_total)[0,:],1)
    
    word_vectors = np.zeros((n_top_words, vectorized_headlines.shape[1]))
    for i in range(n_top_words):
        word_vectors[i,word_indices[0,i]] = 1

    words = [word[0].encode('ascii').decode('utf-8') for 
             word in count_vectorizer.inverse_transform(word_vectors)]

    return (words, word_values[0,:n_top_words].tolist()[0])

raw_data['Labels'] = raw_data['Labels'].fillna('c')
 
# Now, you can use the CountVectorizer as before
count_vectorizer = CountVectorizer(stop_words='english')
words, word_values = get_top_n_words(n_top_words=15,
                                     count_vectorizer=count_vectorizer, 
                                     text_data=raw_data['Labels'])



# Create a bar plot to visualize the top words
fig, ax = plt.subplots(figsize=(20, 8))
ax.bar(range(len(words)), word_values)
top_n = 15  # Choose the number of top words to display
ax.bar(range(top_n), word_values[:top_n])
ax.set_xticks(range(top_n))
ax.set_xticklabels(words[:top_n], rotation=90)
ax.set_xticklabels(words, rotation=90)  # Adjust the rotation to 90 degrees
ax.set_title('Top words in comments dataset (excluding stop words)')
ax.set_xlabel('Word')
ax.set_ylabel('Number of occurrences')
plt.show()

import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer  # Import the WordNetLemmatizer

# Tokenize and preprocess the comments
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()  # Create a WordNetLemmatizer instance

def preprocess_text(text):
    tokens = word_tokenize(text)
    tokens = [token.lower() for token in tokens if token.isalpha()]
    tokens = [token for token in tokens if token not in stop_words]
    tokens = [lemmatizer.lemmatize(token) for token in tokens]  # Use lemmatization instead of stemming
    return tokens

raw_data['Labels'] = raw_data['Labels'].apply(preprocess_text)


from gensim.corpora import Dictionary

# Create a Gensim dictionary
dictionary = Dictionary(raw_data['Labels'])

# Create a corpus (bag of words representation)
corpus = [dictionary.doc2bow(comment) for comment in raw_data['Labels']]

from gensim.models import LdaModel

# Number of topics
num_topics = 3  # You can adjust this number

# Train the LDA model
lda_model = LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=15)

# Print the topics and their top words
for topic_id, topic in lda_model.print_topics():
    print(f"Topic {topic_id}: {topic}")

# Define the number of topics (assumed to be 5 in this example)
num_topics = 3

# Create a figure with subplots
fig, axes = plt.subplots(1, num_topics, figsize=(15, 5))

for topic_id in range(num_topics):
    words = lda_model.show_topic(topic_id, topn=10)
    word_list, freq_list = zip(*words)
    
    # Plot in the corresponding subplot
    axes[topic_id].barh(word_list, freq_list, color = '#E52B12')
    axes[topic_id].set_title(f"Topic {topic_id}")
    axes[topic_id].set_xlabel('Frequency')
    axes[topic_id].invert_yaxis()

plt.tight_layout()
plt.show()

doc_topic_distributions = [lda_model[doc] for doc in corpus]


dominant_topics = [max(dist, key=lambda x: x[1]) for dist in doc_topic_distributions]

topic_labels = ["Topic A", "Topic B", "Topic C", "Topic D", "Topic E", "Topic F"]  # Replace with your topic labels
dominant_topic_labels = [topic_labels[topic_id] for (topic_id, _) in dominant_topics]

raw_data['Dominant Topic'] = dominant_topic_labels

doc_topic_distributions

data_dicts = [dict(item) for item in doc_topic_distributions]
df = pd.DataFrame(data_dicts)
df.columns = ["column1", "column2", "column3"]
raw_data.head()


In [ ]:
raw_data = pd.read_csv(r"C:\Users\rsk29\Documents\UT\Fall\AUD\Project\Disneyplus_Image_tags.csv")

raw_data.head()

# Define helper functions
def get_top_n_words(n_top_words, count_vectorizer, text_data):
    '''
    returns a tuple of the top n words in a sample and their 
    accompanying counts, given a CountVectorizer object and text sample
    '''
    vectorized_headlines = count_vectorizer.fit_transform(text_data.values)
    vectorized_total = np.sum(vectorized_headlines, axis=0)
    word_indices = np.flip(np.argsort(vectorized_total)[0,:], 1)
    word_values = np.flip(np.sort(vectorized_total)[0,:],1)
    
    word_vectors = np.zeros((n_top_words, vectorized_headlines.shape[1]))
    for i in range(n_top_words):
        word_vectors[i,word_indices[0,i]] = 1

    words = [word[0].encode('ascii').decode('utf-8') for 
             word in count_vectorizer.inverse_transform(word_vectors)]

    return (words, word_values[0,:n_top_words].tolist()[0])

raw_data['Labels'] = raw_data['Labels'].fillna('c')
 
# Now, you can use the CountVectorizer as before
count_vectorizer = CountVectorizer(stop_words='english')
words, word_values = get_top_n_words(n_top_words=15,
                                     count_vectorizer=count_vectorizer, 
                                     text_data=raw_data['Labels'])



# Create a bar plot to visualize the top words
fig, ax = plt.subplots(figsize=(20, 8))
ax.bar(range(len(words)), word_values)
top_n = 15  # Choose the number of top words to display
ax.bar(range(top_n), word_values[:top_n])
ax.set_xticks(range(top_n))
ax.set_xticklabels(words[:top_n], rotation=90)
ax.set_xticklabels(words, rotation=90)  # Adjust the rotation to 90 degrees
ax.set_title('Top words in comments dataset (excluding stop words)')
ax.set_xlabel('Word')
ax.set_ylabel('Number of occurrences')
plt.show()

import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer  # Import the WordNetLemmatizer

# Tokenize and preprocess the comments
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()  # Create a WordNetLemmatizer instance

def preprocess_text(text):
    tokens = word_tokenize(text)
    tokens = [token.lower() for token in tokens if token.isalpha()]
    tokens = [token for token in tokens if token not in stop_words]
    tokens = [lemmatizer.lemmatize(token) for token in tokens]  # Use lemmatization instead of stemming
    return tokens

raw_data['Labels'] = raw_data['Labels'].apply(preprocess_text)


from gensim.corpora import Dictionary

dictionary = Dictionary(raw_data['Labels'])

corpus = [dictionary.doc2bow(comment) for comment in raw_data['Labels']]

from gensim.models import LdaModel


num_topics = 3  # You can adjust this number


lda_model = LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=15)


for topic_id, topic in lda_model.print_topics():
    print(f"Topic {topic_id}: {topic}")

num_topics = 3

fig, axes = plt.subplots(1, num_topics, figsize=(15, 5))

for topic_id in range(num_topics):
    words = lda_model.show_topic(topic_id, topn=10)
    word_list, freq_list = zip(*words)
    
    # Plot in the corresponding subplot
    axes[topic_id].barh(word_list, freq_list, color = '#000c7c')
    axes[topic_id].set_title(f"Topic {topic_id}")
    axes[topic_id].set_xlabel('Frequency')
    axes[topic_id].invert_yaxis()

plt.tight_layout()
plt.show()


doc_topic_distributions = [lda_model[doc] for doc in corpus]


dominant_topics = [max(dist, key=lambda x: x[1]) for dist in doc_topic_distributions]

topic_labels = ["Topic A", "Topic B", "Topic C", "Topic D", "Topic E", "Topic F"]  # Replace with your topic labels
dominant_topic_labels = [topic_labels[topic_id] for (topic_id, _) in dominant_topics]


raw_data['Dominant Topic'] = dominant_topic_labels


data_dicts = [dict(item) for item in doc_topic_distributions]
df = pd.DataFrame(data_dicts)
df.columns = ["column1", "column2", "column3"]
